In [1]:
import anchorpy
import solana
from dotenv import load_dotenv
import os
import sys
from solders.pubkey import Pubkey
from solana.rpc.api import Client
from solders.hash import Hash
from solders.keypair import Keypair
from solders.message import MessageV0
from solders.system_program import TransferParams, transfer
from solders.transaction import VersionedTransaction
from solders.instruction import AccountMeta, Instruction;
import borsh_construct as borsh;
from solders.address_lookup_table_account import AddressLookupTableAccount, AddressLookupTable
import typing
from construct import Construct
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [2]:
load_dotenv()
client = Client("http://127.0.0.1:8899")
print(client.is_connected())
sender = Keypair.from_base58_string(str(os.getenv("alice")))

True


In [3]:
from scope.instructions import refreshPriceList
ix = refreshPriceList.RefreshPriceList(refreshPriceList.RefreshPriceListArgs(tokens=[22,0]),
                                  refreshPriceList.RefreshPriceListAccounts({
                                    "oraclePrices":Pubkey.from_string("3NJYftD5sjVfxSnUdZ1wVML8f3aC6mp1CXCL6L7TnU8C"),
                                    "oracleMappings":Pubkey.from_string("Chpu5ZgfWX5ZzVpUx9Xvv4WPM75Xd7zPJNDPsFnCpLpk"),
                                    "oracleTwaps":Pubkey.from_string("GbpsVomudPRRwmqfTmo3MYQVTikPG6QXxqpzJexA1JRb"),
                                    "instructionSysvarAccountInfo":Pubkey.from_string("Sysvar1nstructions1111111111111111111111111"),}),
                                    remaining_accounts=[
                                        AccountMeta(pubkey=Pubkey.from_string("HT2PLQBcG5EiCcNSaMHAjSgd9F98ecpATbk4Sk5oYuM"), is_signer=False, is_writable=True),
                                        AccountMeta(pubkey=Pubkey.from_string("7UVimffxr9ow1uXYxsr4LHAcV58mLzhmwaeKvJ1pjLiE"), is_signer=False, is_writable=True),
                                    ])
alts = [Pubkey.from_string("284iwGtA9X9aLy3KsyV8uT2pXLARhYbiSi5SiM2g47M2")]
account_info =  client.get_multiple_accounts(alts).value    
address_table_list = [
                AddressLookupTableAccount(
                    key=k,
                    addresses=AddressLookupTable.deserialize(
                        account_info[i].data
                    ).addresses
                )
                for i, k in enumerate(alts)
            ]
blockhash = client.get_latest_blockhash().value.blockhash #Hash.default()  # replace with a real blockhash using get_latest_blockhash
msg = MessageV0.try_compile(
        payer=sender.pubkey(),
        instructions=[ix],
        address_lookup_table_accounts=address_table_list,
        recent_blockhash=blockhash,
    )
tx = VersionedTransaction(msg, [sender])
result = client.simulate_transaction(tx) 
print(result.value.logs)


['Program HFn8GnPADiny6XqUoWE8uRPPxb29ikn4yTuPa9MF2fWJ invoke [1]', 'Program log: Instruction: RefreshPriceList', 'Program log: tk 22, PythPull: 100046206 to 100046206 | prev_slot: 343780388, new_slot: 343780355, crt_slot: 343780690', 'Program log: tk 0, PythPull: 15764440440 to 15765390000 | prev_slot: 343780390, new_slot: 343780400, crt_slot: 343780690', 'Program HFn8GnPADiny6XqUoWE8uRPPxb29ikn4yTuPa9MF2fWJ consumed 15591 of 200000 compute units', 'Program HFn8GnPADiny6XqUoWE8uRPPxb29ikn4yTuPa9MF2fWJ success']
